In [1]:
using Iterators
using DataStructures
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util"                                                        

In [2]:
using Packing

In [3]:
using DataStructures

function Base.sum(acc::Accumulator)
    sum(values(acc.map))
end

function Base.sum(acc::Dict)
    sum(values(acc))
end

function freq2prob{T,V<:Number}(acc::Union{Accumulator{T,V},Dict{T,V}})
    
    ret=Dict{T,Float64}()
    total = sum(acc)
    for (k,v) in acc
        ret[k]=v/total
    end
    ret
end

freq2prob (generic function with 1 method)

In [4]:
function clean(ss)
    @pipe (ss 
    |> replace(_, r"\[.*?\] ?","")  #Remove Nonword sounds
    |> replace(_, r"\<.*?\> ","")  #Remove Verbal Deltions
    |> replace(_, r"\*(.*?)\*",s"\1") #Remove mispronounciation marks
    |> replace(_,r"\:|\-\s\.\s\-", "") #remove intraword pauses
    |> replace(_,r"\w+\- ","") #remove stuttered words
    |> replace(_, r"[!\.,\?]","")        #Remove punctation as it is not used traditionally (see sro spec)
    
    |> replace(_, r"\s+",' ') #Remove repeated spaces
    #|> replace(_, r"([A-Z])\s([A-Z])", s"\1\2") #Merge len(2) abbrev
    |> replace(_, r".*[~\(\)\-\<\>#'].*","")#Remove everything if anything unfixable found
    |> lowercase
    |> strip 
    )
end


clean (generic function with 1 method)

In [5]:
const START_MARKER = "**START**"
const END_MARKER = "**END**"

function collect_grams_stats(sentences)
    unigrams = counter(AbstractString)
    bigrams = DefaultDict(()->counter(AbstractString))
    
    for sent in sentences
        push!(bigrams[START_MARKER], sent[1])
        for ii in 1:length(sent)-1
            push!(unigrams,sent[ii])
            push!(bigrams[sent[ii]], sent[ii+1])    
        end
        push!(unigrams,sent[end])
        push!(bigrams[sent[end]], END_MARKER)
    end
    
    
    [k=>v.map for (k,v) in bigrams], unigrams.map
end




collect_grams_stats (generic function with 1 method)

In [6]:
path="../../Resources/corpora/atis2_text/"
function valid(ss)
    typeof(ss) <: ASCIIString  && length(ss)>0
end
    
corpus = @pipe readdir(path) |> filter!(fn -> splitext(fn)[2]==".sro", _) |> map(_) do fn
    try open(readall, path*fn) end
    end |> filter!(valid,_) |> map(clean,_) |> filter!(valid,_) |> map(s->split(s),_);
corpus_vocab = @pipe corpus |> map(Set,_) |> reduce(union,_)
length(corpus_vocab)

1131

In [7]:
using WordEmbeddings
LL, word_indexes, indexed_words = load_word2vec_embeddings("word_emb_data/GoogleNews-vectors-negative300.bin", length(corpus_vocab), corpus_vocab);

In [8]:
setdiff(corpus_vocab,indexed_words)


16-element Array{UTF8String,1}:
 "and"      
 "a"        
 "respeak"  
 "nondirect"
 "maluso"   
 "of"       
 "panam"    
 "stapleton"
 "fokker"   
 "lufthansa"
 "to"       
 "laguardia"
 "nonjets"  
 "hartfield"
 "mcdonnell"
 "dulles"   

In [9]:
#Kind of the opposite of a stop word. This word has little meaning (So zero value), but much structural importance
forcewords = ["and", "a", "of", "to"]
for word in forcewords
    @assert(!(word in indexed_words))
    push!(indexed_words, word)
    word_indexes[word] = length(indexed_words)
end
LL = [LL zeros(size(LL,1),length(forcewords))]

300x1119 Array{Float64,2}:
  0.0529562  -0.00851202  -0.0123606    …  -0.0548293   0.0  0.0  0.0  0.0
  0.0654598  -0.0342245   -0.0222299        0.0239878   0.0  0.0  0.0  0.0
  0.0661953   0.0322839    0.0655398       -0.0190492   0.0  0.0  0.0  0.0
  0.0470722   0.0458679    0.0394772        0.0826471   0.0  0.0  0.0  0.0
  0.0522207  -0.0131429   -0.0866199        0.0620861   0.0  0.0  0.0  0.0
 -0.0820086  -0.0462207    0.0249128    …   0.0465646   0.0  0.0  0.0  0.0
 -0.0614145  -0.00094823  -0.0111628       -0.0786156   0.0  0.0  0.0  0.0
 -0.11621    -0.0522188   -0.0705224       -0.0516041   0.0  0.0  0.0  0.0
  0.0156294   0.0465735    0.092369         0.00902063  0.0  0.0  0.0  0.0
  0.0992929   0.0624509    0.0927522        0.0364857   0.0  0.0  0.0  0.0
 -0.0856861  -0.122785    -0.0563412    …   0.0395094   0.0  0.0  0.0  0.0
 -0.028133   -0.0287556   -0.0605572       -0.0588609   0.0  0.0  0.0  0.0
  0.0522207   0.0515131   -0.0540416       -0.0903071   0.0  0.0  0.0  0.

In [10]:
@assert(!(START_MARKER in indexed_words))
push!(indexed_words, START_MARKER)
word_indexes[START_MARKER] = length(indexed_words)

@assert(!(END_MARKER in indexed_words))
push!(indexed_words, END_MARKER)
word_indexes[END_MARKER] = length(indexed_words)
LL = [LL zeros(size(LL,1),2)]

300x1121 Array{Float64,2}:
  0.0529562  -0.00851202  -0.0123606    …  0.0  0.0  0.0  0.0  0.0  0.0
  0.0654598  -0.0342245   -0.0222299       0.0  0.0  0.0  0.0  0.0  0.0
  0.0661953   0.0322839    0.0655398       0.0  0.0  0.0  0.0  0.0  0.0
  0.0470722   0.0458679    0.0394772       0.0  0.0  0.0  0.0  0.0  0.0
  0.0522207  -0.0131429   -0.0866199       0.0  0.0  0.0  0.0  0.0  0.0
 -0.0820086  -0.0462207    0.0249128    …  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0614145  -0.00094823  -0.0111628       0.0  0.0  0.0  0.0  0.0  0.0
 -0.11621    -0.0522188   -0.0705224       0.0  0.0  0.0  0.0  0.0  0.0
  0.0156294   0.0465735    0.092369        0.0  0.0  0.0  0.0  0.0  0.0
  0.0992929   0.0624509    0.0927522       0.0  0.0  0.0  0.0  0.0  0.0
 -0.0856861  -0.122785    -0.0563412    …  0.0  0.0  0.0  0.0  0.0  0.0
 -0.028133   -0.0287556   -0.0605572       0.0  0.0  0.0  0.0  0.0  0.0
  0.0522207   0.0515131   -0.0540416       0.0  0.0  0.0  0.0  0.0  0.0
  ⋮                                  

In [11]:
known_vocab = Set(indexed_words)
known_corpus = filter(corpus) do sent
    for word in sent
        if !( word in known_vocab)
            return false
        end
    end
    true
end;

In [ ]:
################LOADING Done, Now processing

In [ ]:
using StatsBase
# modified from https://github.com/JoFrhwld/GoodTuring.jl/blob/master/GoodTuring.jl
function simpleGoodTuring(speciesCountDict::Dict)
    speciesCountVec = collect(values(speciesCountDict))
        
    totalCounts = sum(speciesCountVec)
    cofcDict = countmap(speciesCountVec)
    r = sort(collect(keys(cofcDict)))

    N = size(r,1)
    Nr = [cofcDict[r[i]] for i in 1:N]

    p0 = haskey(cofcDict, 1.0) ? cofcDict[1.0] / totalCounts : 0.0
    
    Z = sgtZ(r,Nr)
    logr = map(log,r)
    logZ = map(log,Z)

    X = hcat(ones(N), logr)
    Y = copy(logZ )
    coefs = X\Y
    intercept = coefs[1]
    slope = coefs[2]

    useY = false
    rSmooth = Array{Float64}(N)
    for i in 1:N
        @inbounds thisr = r[i]
        
        #y = ((thisr+1.0)^(slope+1.0))/(thisr^slope)
        #The above is the much simplified form of the below (Performance identical output differs by 10^-16)
        y = (thisr+1.0) * exp(slope * log(thisr+1.0) + intercept) / exp(slope * log(thisr) + intercept)

        if !in(thisr+1, r)
            useY = true
        end

        if useY
            rSmooth[i] = y
        else
            x = (thisr+1) * cofcDict[thisr + 1]/cofcDict[thisr]
            thisNr = cofcDict[thisr]
            thisNr1 = cofcDict[thisr+1]

            t = 1.96 * ((thisr+1)^2) * (thisNr1 / thisNr^2) * (1 + (thisNr1 / thisNr))

            if abs(x-y) > t
                @inbounds rSmooth[i] = x
            else
                useY = true
                @inbounds rSmooth[i] = y
            end
        end
    end

    smoothTot = sum(Nr.*rSmooth)
    sgtProb  = (1.0 - p0) .* (rSmooth/smoothTot)
    sgtProbDict = Dict([r[i] => sgtProb[i] for i in 1:N])
    sgtDict = Dict([sp=>sgtProbDict[speciesCountDict[sp]] for sp in keys(speciesCountDict)])

    sgtDict, sgtProbDict, p0
end


function sgtZ(r::Array, Nr::Array)
    j = r
    i = [0; j[1:end-1]]
    lastK = 2*j[end] - i[end]
    k = [j[2:end]; lastK]
    Float64[(2*Nr[iter])/(k[iter]-i[iter]) for iter = 1:length(j)]
end

function simpleGoodTuring(speciesCountVec::Vector)
    sgtD = simpleGoodTuring(Dict([ii=>v for (ii,vv) in enumerate(speciesCountVec)]))
    [sgtD[ii] for ii in 1:length(speciesCountVec)]
end

In [ ]:
function katz_bigrams(bigram_freq::Dict, unigrams_freq::Dict)
    k_bigrams = Dict()
    
    for first in keys(bigram_freq)
        smoothed,_,p0 = simpleGoodTuring(bigram_freq[first])
        k_bigrams[first] = smoothed
        
        backoff_keys = setdiff(keys(unigram_freq),keys(smoothed))
        #share the p0 proability mass between them
        total = sum([unigrams_freq[key] for key in backoff_keys])
        for second in backoff_keys
            k_bigrams[first][second]=p0.*unigrams_freq[second]./total
        end
    end
    k_bigrams
end

In [ ]:
Vector

In [ ]:
function dict2mat(bigrams::Dict, word_indexes::Dict{AbstractString,Int64}, dense=False)
    mat  = (dense ? zeros: spzeros)(length(word_indexes),length(word_indexes))
    for first in keys(bigrams)
        for second in keys(bigrams[first])
            mat[word_indexes[second], word_indexes[first]] = bigrams[first][second]
        end
    end
    mat
end

In [ ]:
bigram_freq, unigram_freq = collect_grams_stats(known_corpus);
kbigrams=katz_bigrams(bigram_freq, unigram_freq)
kbigrams_mat = dict2mat(kbigrams,word_indexes,true)


In [ ]:
using Gadfly
using Distributions

In [ ]:
sent_lengths = map(length, known_corpus)
plot(x=sent_lengths, Geom.histogram)

In [ ]:
sent_length_dist = fit_mle(Gamma, sent_lengths)
plot(x=[round(rand(sent_length_dist)) for _ in 1:length(sent_lengths)], Geom.histogram)

In [ ]:
length_prob=cdf(sent_length_dist,[1.5:1.0:50.5])-cdf(sent_length_dist,[0.5:1.0:49.5])


In [ ]:
function collect_cooccur_stats(sentences)
    unioccur = counter(AbstractString)
    bioccur = DefaultDict(()->counter(AbstractString))
    
    for sent in sentences
        for ii in 1:length(sent)
            push!(unioccur, sent[ii])
            for jj in 1:length(sent)
                if ii==jj
                    continue
                end
                push!(bioccur[sent[ii]], sent[jj])    
            end
        end       
    end
    
    [k=>v.map for (k,v) in bioccur], unioccur.map
    
end

In [ ]:
bioccur_freq, unioccur_freq = collect_cooccur_stats(known_corpus)

bioccur_mat = dict2mat(bioccur_freq,word_indexes,true)
bioccur_mat.+=1.0 # Add one smoothing
bioccur_mat./=sum(bioccur_mat)

unioccur = freq2prob(unioccur_freq)
unioccur_vec = Float64[word in keys(unioccurs) ? unioccurs[word] : 0.0 for word in indexed_words]

In [ ]:
unioccur_vec_smoothed = Float64[word in keys(unioccur_freq) ? unioccur_freq[word] : 0.0 for word in indexed_words]
unioccur_vec_smoothed.+=1.0 # Add one smoothing
unioccur_vec_smoothed./=sum(unioccur_vec_smoothed)

In [ ]:
sum(bioccur_mat,2)

In [ ]:
open("atis_data.jsz","w") do fh
    data = Dict([
        ("bigrams", kbigrams_mat),
        ("bioccur", bioccur_mat),
        ("unioccur", unioccur_vec),
        
        ("length_prob", length_prob),
        ("LL",LL),
        ("word_indexes", word_indexes),
        ("indexed_words", indexed_words),
        ])
    serialize(fh, data)    
end

In [ ]:
function likelyhood(sent, bigrams)
    words = split(sent)
    words = [START_MARKER; words; END_MARKER]
    
    p=1.0
    for ii in 1:length(words)-1
        p*=bigrams[words[ii]][words[ii+1]]
    end
    p
end


function select_word{S<:AbstractString,V}(unigrams::Dict{S,V})
    cutoff = rand()
    total = 0.0
    for next_word in keys(unigrams)
        total+=unigrams[next_word]
        if total>=cutoff
            return next_word
        end
    end
    assert(False, "Should never reach here") 
end

function random_walk(bigrams)
    words=[]
    cur = START_MARKER
    while(cur!=END_MARKER)
        cur = select_word(bigrams[cur])
        push!(words,cur)
    end
    words = words[1:end-1]
    join(words, " ")
end

walk =random_walk(kbigrams) 
print(walk*"\t")
print(likelyhood(walk,kbigrams))

In [ ]:
LL

Frames Magic Sharing Co-occurance PMF, Inspired by Bengio 2003
---

In [12]:
dEmb = 32
CC = 0.01*randn((dEmb, length(indexed_words)))
WW = 0.01*randn((length(indexed_words),dEmb))
bb = 0.01*randn((length(indexed_words)));

In [13]:
@pz CC

CC		

In [14]:
function evaluate(sentence)
    sentence_iis = [word_indexes[word] for word in sentence]

    for n_givens in 1:length(sentence_iis)
        given_prob = 2* 1.0/n_givens
        given_keep = rand(length(sentence_iis)).<given_prob
        given_iis=  sentence_iis[given_keep]
        cooccur_iis =  sentence_iis[~given_keep]
        given_sowe = sum(CC[:,given_iis],2)
        return given_sowe, cooccur_iis
        input_iis
    end
    
        
end

evaluate (generic function with 1 method)

In [37]:
function loss(actual, expected)
    sum(0.5*(expected-actual).^2)
end

function forward(x,W,b)
    tanh(WW*x+b)
end

function δ(a, δ_above, W)
    #a is the ouput of this layer: a=tanh(z) where z is the input from layer below
    #W is matrix to move to above layer, from this one
    const dz = 1-a.^2 #Derivitive of a=tanh(z)
    (W'*δ_above).*dz
end

function δ(actual, expected) 
    #Output Layer
    const M = length(actual)# ==length(expected)
    const dz = 1-actual.^2
    const δ_above = -(expected-actual)
    δ_above.*dz
end

function train(input,W,b, expected_output)
    actual_output = forward(input,W,b)
    δ_top = δ(actual_output, expected_output)
    ΔW  = δ_top*actual_output'
    Δb  = δ_top
    δ_bottom = δ(input, δ_top, W)
    Δx  = δ_bottom
    Δx,ΔW,Δb
end


    

train (generic function with 1 method)

In [16]:
x,t_iis=evaluate(["shortest", "flight"])
target = zeros(bb) #just while we are testing use a one hot bag rep
target[t_iis]=1.0;

In [46]:
train(x,WW,bb,target)

(
32x1 Array{Float64,2}:
 -0.000867156
  0.00313226 
  0.00310955 
  0.00641466 
  0.000592477
 -0.00519664 
 -0.00149645 
 -8.56412e-5 
  0.0037847  
 -0.00371514 
  0.00364238 
  0.00300397 
  0.00187676 
  ⋮          
  0.00325599 
 -0.00169455 
 -0.00141891 
 -0.00579617 
  0.00347054 
  0.00469104 
 -0.0050027  
 -0.00414357 
  0.00634323 
 -0.00441365 
 -0.00355065 
 -0.00642642 ,

1121x1121 Array{Float64,2}:
  3.1688e-5     3.64543e-5    3.16231e-5   …  -3.80822e-5    3.08786e-5 
  3.64539e-5    4.19371e-5    3.63793e-5      -4.38099e-5    3.55229e-5 
  3.16231e-5    3.63797e-5    3.15584e-5      -3.80043e-5    3.08154e-5 
  1.61214e-5    1.85463e-5    1.60884e-5      -1.93745e-5    1.57097e-5 
 -2.56802e-5   -2.95429e-5   -2.56277e-5       3.08622e-5   -2.50243e-5 
 -5.64577e-6   -6.49498e-6   -5.63422e-6   …   6.78502e-6   -5.50157e-6 
  8.49654e-6    9.77454e-6    8.47915e-6      -1.0211e-5     8.27953e-6 
  3.53348e-5    4.06497e-5    3.52625e-5      -4.2465e-5     3.44324e-

In [35]:
using ForwardDiff

function f(θ)
    actual = forward(unpack(θ,size(x),size(WW),size(bb))...)
    loss(actual, target)
end

# Using forwarddiff_jacobian
g = forwarddiff_gradient(f, Float64, fadtype=:dual, n=length(pack(x,WW,bb)))

g (generic function with 1 method)

In [43]:
t=pack(x,WW,bb)
dg = g(t)

37025-element Array{Float64,1}:
 -0.000867328
  0.00313256 
  0.00310955 
  0.00641479 
  0.000592594
 -0.00519668 
 -0.00149691 
 -8.56524e-5 
  0.00378514 
 -0.00371579 
  0.00364245 
  0.003004   
  0.00187687 
  ⋮          
  0.00654075 
  0.0133394  
 -0.00615083 
  0.0113756  
  0.00457073 
 -0.00012692 
  0.00916631 
 -0.0168069  
 -0.0243315  
  0.00277478 
 -0.00676491 
  0.00548535 

In [42]:
ag = pack(train(x,WW,bb, target)...)

1257794-element Array{Float64,1}:
 -0.000867156
  0.00313226 
  0.00310955 
  0.00641466 
  0.000592477
 -0.00519664 
 -0.00149645 
 -8.56412e-5 
  0.0037847  
 -0.00371514 
  0.00364238 
  0.00300397 
  0.00187676 
  ⋮          
  0.00654075 
  0.0133394  
 -0.00615083 
  0.0113756  
  0.00457073 
 -0.00012692 
  0.00916631 
 -0.0168069  
 -0.0243315  
  0.00277478 
 -0.00676491 
  0.00548535 

In [26]:
`git add ../util/Packing.jl` |> run

In [45]:
abs(dg.-ag)

LoadError: LoadError: DimensionMismatch("arrays could not be broadcast to a common size")
while loading In[45], in expression starting on line 1

In [ ]:
`git commit -m="Working on neural cooccuance measure -a` |> run